In [3]:

from mindquantum.algorithm.nisq import HardwareEfficientAnsatz,RYFull
from mindquantum.core.parameterresolver import  PRGenerator
import numpy as np
from mindquantum.core.gates import RX, RY, RZ, H, X, Y, Z, I,CNOT
from mindquantum.core.circuit import Circuit,UN
import mindspore as ms
import pickle
from mindquantum.core.parameterresolver import PRGenerator
import random
from mindspore import Tensor,ops
import tensorcircuit as tc
import tensorflow as tf
import mindspore.numpy as mnp
from DQAS_tool import wash_pr,Mindspore_ansatz_micro,best_from_structure
from DQAS_tool import  sampling_from_structure,zeroslike_grad_nnp_micro_minipool,nmf_gradient,vag_nnp_micro,DQAS_accuracy,Washing_namemap
import sys
from typing import Union
sys.path.append('..')
from Test_tool import Test_ansatz
from data_processing import X_train,X_test,y_train,y_test
from mindquantum.core.circuit import change_param_name,apply

pr_pool = PRGenerator('pool')
parameterized_circuit= \
[
 UN(RZ(pr_pool.new()),maps_obj=[0])+\
 UN(RY(pr_pool.new()),maps_obj=[0])+\
 UN(RZ(pr_pool.new()),maps_obj=[0])+I.on(1),
 UN(RZ(pr_pool.new()),maps_obj=[1])+\
 UN(RY(pr_pool.new()),maps_obj=[1])+\
 UN(RZ(pr_pool.new()),maps_obj=[1])+I.on(0),]


unparameterized_circuit = \
[UN(X,maps_obj=[0],maps_ctrl=[1]),
 UN(X,maps_obj=[1],maps_ctrl=[0]),
 ]
ansatz_pr = PRGenerator('ansatz')
shape_parametized = len(parameterized_circuit)
shape_unparameterized = len(unparameterized_circuit)
num_layer=4
shape_nnp = (7,num_layer,shape_parametized,3)
shape_stp = (num_layer,shape_unparameterized+shape_parametized)
stddev = 0.03
np.random.seed(2)
nnp = np.random.normal(loc=0.0, scale=stddev, size=shape_nnp)
stp = np.random.normal(loc=0.0, scale=stddev, size=shape_stp)
ops_onehot = ops.OneHot(axis=-1)

In [5]:
from DQAS_tool import  sampling_from_structure,vag_nnp_micro_minipool,zeroslike_grad_nnp_micro_minipool,nmf_gradient,DQAS_accuracy,Mindspore_ansatz_micro_minipool,nnp_dealwith
#from DQAS_tool import  DQASAnsatz_from_result,DQAS_accuracy
K = tc.set_backend("tensorflow")
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.06, 100, 0.5)
structure_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(0.1))
network_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(lr))
verbose = False
# 设置超参数
epochs = 100
batch_size=100
avcost1 = 0
ops_onehot = ops.OneHot(axis=-1)
batch_loss_history=[] # 记录每个epoch的batch_size损失值
structure_distribution_history=[] # 记录每个epoch的结构参数
ansatz_params_history=[] # 记录每个epoch的网络参数
best_candidates_history=[] # 记录每个epoch的最佳候选
acc_history = [] #记录每个epoch的准确率

 
for epoch in range(epochs):  # 更新结构参数的迭代
    avcost2 = avcost1
    costl = []
    tmp = np.stack([sampling_from_structure(stp,num_layer,shape_parametized) for _ in range(batch_size)])
    batch_structure = ops_onehot(ms.Tensor(tmp),shape_parametized+shape_unparameterized,ms.Tensor(1),ms.Tensor(0))
    #print(batch_structure.shape)
    # print(tmp,batch_structure)
    loss_value = []
    grad_nnps = []
    grad_stps = []
    
    for i in batch_structure:
        #print(ops.Argmax()(i))          
        infd, grad_nnp = vag_nnp_micro_minipool(Structure_params=i,
                                    Ansatz_params=nnp,
                                    paramerterized_pool=parameterized_circuit,  unparamerterized_pool=unparameterized_circuit,
                                    num_layer=num_layer,n_qbits=8)(ms.Tensor(X_train),ms.Tensor(y_train))
        
        grad_nnp_zeroslike = zeroslike_grad_nnp_micro_minipool(batch_sturcture=i,grad_nnp=grad_nnp[0],shape_parametized=shape_parametized,ansatz_parameters=nnp)
        gs = nmf_gradient(structures=stp,oh=i,num_layer=num_layer,size_pool=stp.shape[1])
        #print(infd,grad_nnp)
        loss_value.append(infd)
        grad_nnps.append(ms.Tensor(grad_nnp_zeroslike,dtype=ms.float64))
        grad_stps.append(gs)

      
    infd = ops.stack(loss_value)
    gnnp = ops.addn(grad_nnps)
    gstp = [(infd[i] - avcost2) * grad_stps[i] for i in range(infd.shape[0])]
    gstp_averge = ops.addn(gstp) / infd.shape[0]
    avcost1 = sum(infd) / infd.shape[0]
    # print(f'loss={infd}\ngrad_nnp={gnnp}\ngrandient_stp={gstp_averge}')
    
    gnnp_tf = tf.convert_to_tensor(gnnp.asnumpy(),dtype=tf.float64)
    nnp_tf = tf.convert_to_tensor(nnp,dtype=tf.float64)
    gstp_averge_tf = tf.convert_to_tensor(gstp_averge.reshape(stp.shape).asnumpy(),dtype=tf.float64)
    stp_tf = tf.convert_to_tensor(stp,dtype=tf.float64)
     # 更新参数
    nnp_tf = network_opt.update(gnnp_tf, nnp_tf)
    stp_tf = structure_opt.update(gstp_averge_tf, stp_tf) 
    
    nnp = nnp_tf.numpy()
    stp = stp_tf.numpy()

    batch_loss_history.append(avcost1)
    structure_distribution_history.append(stp)
    ansatz_params_history.append(nnp)
    #best_candidates_history.append(best_from_structure(cand_preset.asnumpy()))
    cand_preset = best_from_structure(stp)
    best_candidates_history.append(cand_preset.asnumpy())
    

    if epoch % 1 == 0 or epoch == epochs - 1:
        print("----------epoch %s-----------" % epoch)
        print(
            "batched平均损失: ",
            avcost1,
        )
    
        if verbose:
            print(
                "strcuture parameter: \n",
                stp,
                "\n network parameter: \n",
                nnp,
            )
        
        print("最好的候选结构:",cand_preset)
        stp_for_test = ops_onehot(ms.Tensor(cand_preset),shape_parametized+shape_unparameterized,ms.Tensor(1),ms.Tensor(0))

        
        if cand_preset.min() <shape_parametized:
            ansatz_parameters = nnp_dealwith(Structure_params=stp_for_test,Network_params=nnp)
            test_ansatz = Mindspore_ansatz_micro_minipool(Structure_p=stp_for_test,
                                            parameterized_pool=parameterized_circuit,unparameterized_pool=unparameterized_circuit,
                                            num_layer=num_layer,
                                            n_qbits=8)
            acc = DQAS_accuracy(ansatz=test_ansatz,Network_params=ansatz_parameters,n_qbits=8)
            acc_history.append(acc)
            print(f'二分类准确率 Acc ={acc*100}% ')
        
        #我想每一轮结束 保存batch_loss_history、structure_distribution_history、ansatz_params_history、best_candidates_history、acc_history
                # 保存数据
        with open('training_history-minipool-k4.pkl', 'wb') as f:
            pickle.dump({
                'batch_loss_history': batch_loss_history,
                'structure_distribution_history': structure_distribution_history,
                'ansatz_params_history': ansatz_params_history,
                'best_candidates_history': best_candidates_history,
                'acc_history': acc_history
            }, f)
        
        

----------epoch 0-----------
batched平均损失:  0.8989891
最好的候选结构: [2 2 3 3]
----------epoch 1-----------
batched平均损失:  0.8224533
最好的候选结构: [2 2 3 3]
----------epoch 2-----------
batched平均损失:  0.806179
最好的候选结构: [2 2 3 3]
----------epoch 3-----------
batched平均损失:  0.7906138
最好的候选结构: [2 2 3 3]
----------epoch 4-----------
batched平均损失:  0.7756768
最好的候选结构: [2 2 3 3]
----------epoch 5-----------
batched平均损失:  0.74004054
最好的候选结构: [2 2 3 3]
----------epoch 6-----------
batched平均损失:  0.7490204
最好的候选结构: [2 2 3 3]
----------epoch 7-----------
batched平均损失:  0.7249916
最好的候选结构: [2 2 3 3]
----------epoch 8-----------
batched平均损失:  0.66899323
最好的候选结构: [2 2 3 3]
----------epoch 9-----------
batched平均损失:  0.67487985
最好的候选结构: [2 2 3 3]
----------epoch 10-----------
batched平均损失:  0.65214455
最好的候选结构: [2 2 3 3]
----------epoch 11-----------
batched平均损失:  0.65023583
最好的候选结构: [2 2 3 3]
----------epoch 12-----------
batched平均损失:  0.6769554
最好的候选结构: [2 2 3 0]
二分类准确率 Acc =49.94807892004154% 
----------epoch 13-------

由于 DQAS 在量子结构搜索的过程中进行结构、参数的双优化.  
我们可以在结构基本不再变动时停止搜索过程转为单优化 效率更高